# Regresión lineal  

1. Se utilizarán las librerías necesarias para la manipulación de los datos
2. Se abrira el archivo donde se encuentran alojados los datos y se realizará una limpieza
3. Se trabajará con los datos  

Los pasos 1 y 2 se realizan para la regresión lineal simple y la autoregresión

In [75]:
import csv
import numpy

In [76]:
path_pm25 = 'C:/Users/Ian/Documents/PythonDataProducts4PredictiveAnalytics/DesignThinking&PredictiveAnalytics4DataProducts/Final_Course2/datasets/PRSA_data_2010.1.1-2014.12.31.csv'
file = open(path_pm25, 'r')

Encabezado de los datos así como los índices de los datos que son de nuestro interés

In [77]:
dataset = []
header = file.readline().strip().split(',')
for line in file:
    line = line.split(',')
    dataset.append(line)
    
print(header)

print("Índice del dato pm2.5: ", header.index('pm2.5'))
print("Índice del dato de la temperatura: ", header.index('TEMP'))

['No', 'year', 'month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir']
Índice del dato pm2.5:  5
Índice del dato de la temperatura:  7


Limpieza de los datos

In [78]:
dataset = [d for d in dataset if d[5] != 'NA']

## Regresión lineal simple

Utilizamos una característica unicamente, la **temperatura**  

Construcción de la **matriz X de características** así como el vector **y**

In [45]:
def feature(datnum):
    feat = [1, float(datnum[7])]
    return feat

X = [feature(d) for d in dataset]
y = [float(d[5]) for d in dataset]

Ejemplo de los datos

In [46]:
print(y[:10])
print(X[:10])

[129.0, 148.0, 159.0, 181.0, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0]
[[1, -4.0], [1, -4.0], [1, -5.0], [1, -5.0], [1, -5.0], [1, -6.0], [1, -6.0], [1, -5.0], [1, -6.0], [1, -5.0]]


Recordemos que la expresión queda de la siguiente manera  
$pm2.5 = \theta_0 + \theta_1 * temp$

In [47]:
theta, residuals, rank, s = numpy.linalg.lstsq(X,y)
print(theta)

[107.10183392  -0.68447989]


<ipython-input-47-1e07fe70644f>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, residuals, rank, s = numpy.linalg.lstsq(X,y)


Por lo tanto, la línea de regresión resulta ser la siguiente  
$pm2.5 = 107.10183392 - 0.68447989 * temp$

Realizamos las operaciones de forma manual utilizando la expresión  
$ (X^TX)^{-1} X^Ty$

In [48]:
X_matrix = numpy.matrix(X)
y_matrix = numpy.matrix(y)
numpy.linalg.inv(X_matrix.T * X_matrix) * X_matrix.T * y_matrix.T

matrix([[107.10183392],
        [ -0.68447989]])

### Regresión lineal con muchas características

Ahora utilizamos la **Temperatura, Presión y Velocidad del viento**

In [57]:
def multipleFeature(datnum):
    feat = [1, float(datnum[7]), float(datnum[8]), float(datnum[10])]
    return feat


X = [multipleFeature(d) for d in dataset]
y = [float(d[5]) for d in dataset]

Recordemos que la expresión queda de la siguiente manera  
$pm2.5 = \theta_0 + \theta_1 * temp + \theta_1 * press + \theta_1 * windSpeed$

In [58]:
theta, residuals, rank, s = numpy.linalg.lstsq(X,y)
print(theta)

[ 3.26373064e+03 -3.10933772e+00 -3.06517728e+00 -4.60017221e-01]


<ipython-input-58-1e07fe70644f>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, residuals, rank, s = numpy.linalg.lstsq(X,y)


## Extrayendo características de autoregresión
La autoregresión se utiliza para valores donde está presente un intervalo de tiempo en el eje X (variable independiente)  

In [70]:
def featureAR(dataset, ind, windowSize):
    feat = [1]
    previousValues = [float(d[5]) for d in dataset[ind - windowSize: ind]]
    
    return feat + previousValues

windowSize = 10
N = len(dataset)

In [71]:
X = [featureAR(dataset, ind, windowSize) for ind in range(windowSize,N)]
X[:10]

[[1, 129.0, 148.0, 159.0, 181.0, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0],
 [1, 148.0, 159.0, 181.0, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0, 140.0],
 [1, 159.0, 181.0, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0, 140.0, 152.0],
 [1, 181.0, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0, 140.0, 152.0, 148.0],
 [1, 138.0, 109.0, 105.0, 124.0, 120.0, 132.0, 140.0, 152.0, 148.0, 164.0],
 [1, 109.0, 105.0, 124.0, 120.0, 132.0, 140.0, 152.0, 148.0, 164.0, 158.0],
 [1, 105.0, 124.0, 120.0, 132.0, 140.0, 152.0, 148.0, 164.0, 158.0, 154.0],
 [1, 124.0, 120.0, 132.0, 140.0, 152.0, 148.0, 164.0, 158.0, 154.0, 159.0],
 [1, 120.0, 132.0, 140.0, 152.0, 148.0, 164.0, 158.0, 154.0, 159.0, 164.0],
 [1, 132.0, 140.0, 152.0, 148.0, 164.0, 158.0, 154.0, 159.0, 164.0, 170.0]]

In [72]:
y = [float(d[5]) for d in dataset[windowSize:]]

In [73]:
theta, residuals, rank, s = numpy.linalg.lstsq(X,y)
theta

<ipython-input-73-d76bec25dcff>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, residuals, rank, s = numpy.linalg.lstsq(X,y)


array([ 3.92354307,  0.01296079,  0.00414674, -0.00925187,  0.00794837,
       -0.01458271, -0.0164066 ,  0.00626613,  0.04355583, -0.22994129,
        1.15548897])

### Múltiples características

In [84]:
def multipleFeatureAR(dataset, ind, windowSize):
    feat = [1, float(dataset[ind][7]), float(dataset[ind][8]), float(dataset[ind][10])]
    previousValues = [float(d[5]) for d in dataset[ind - windowSize: ind]]
    
    return feat + previousValues

In [85]:
X = [multipleFeatureAR(dataset, ind, windowSize) for ind in range(windowSize,N)]
y = [float(d[5]) for d in dataset[windowSize:]]

In [86]:
theta, residuals, rank, s = numpy.linalg.lstsq(X,y)
theta

<ipython-input-86-d76bec25dcff>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, residuals, rank, s = numpy.linalg.lstsq(X,y)


array([ 1.66261451e+02, -1.71855736e-01, -1.56441424e-01, -2.50471317e-02,
        1.50765140e-02,  3.66638827e-03, -9.37757357e-03,  7.43748773e-03,
       -1.50878276e-02, -1.67895315e-02,  5.71176519e-03,  4.27640539e-02,
       -2.29558932e-01,  1.15031328e+00])